In [8]:
import pandas as pd
import numpy as np
import os
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler, Subset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torch.optim as optim
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Path to the directory containing the plant folders
base_dir = 'PlantVillage'

# Initialize an empty dictionary to store counts of healthy and diseased images for each plant type
plant_counts = {}

# Initialize a list to store the class labels
labels = []

# Loop through each folder in the base directory
for folder_name in os.listdir(base_dir):
    labels.append(folder_name)
    folder_path = os.path.join(base_dir, folder_name)
    
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Split the folder name to get the plant type and health status
        parts = folder_name.split('_')
        plant_type = parts[0]
        health_status = parts[1]
        
        # Initialize the dictionary entry if it doesn't exist
        if plant_type not in plant_counts:
            plant_counts[plant_type] = [0, 0]  # [healthy, diseased]
        
        # Recursively count files in the folder and any subfolders
        num_files = sum([len(files) for _, _, files in os.walk(folder_path)])
        
        # Update the count based on the health status
        if health_status == 'healthy':
            plant_counts[plant_type][0] += num_files
        else:
            plant_counts[plant_type][1] += num_files

print("Healthy vs Diseased Image Count per plant: \n", plant_counts)

Healthy vs Diseased Image Count per plant: 
 {'Tomato': [1591, 14421], 'Potato': [152, 2000], 'BellPepper': [1478, 997]}


In [4]:
# Define image transformations for the training and validation datasets
train_transforms = transforms.Compose([transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                                       
                                      ])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# Load the full dataset and get the label for each image based on the directory name
data_dir = 'PlantVillage'
full_dataset = ImageFolder(root=data_dir)
labels = [sample[1] for sample in full_dataset.samples]
print("Number of classes:", len(full_dataset.classes))  # This should return 15

# Create a stratified train-test split with 20% being used for test data.
train_idx, val_idx = train_test_split(range(len(labels)), test_size=0.2, stratify=labels, random_state=42)
train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(full_dataset, val_idx)

# Calculate the class distribution in the training set
train_labels = [labels[i] for i in train_idx]
unique_labels = set(train_labels)
print("Unique labels in training set:", unique_labels)  # This should show indices from 0 to 14

class_counts = np.bincount(train_labels)
print("Class counts in training set:", class_counts)

# Calculate class weights
class_weights = 1.0 / class_counts
class_weights[np.isinf(class_weights)] = 0.0  # Handle potential infinities
class_weights /= np.sum(class_weights)  # Normalize
print("Class weights:", class_weights)

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Calculate sample weights for the training set only
sample_weights = [class_weights_tensor[label].item() for label in train_labels]  # Use train_labels for correct indexing
print("Sample weights length:", len(sample_weights))  # Should match number of training samples

# Create the WeightedRandomSampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

print("Sampler weights shape:", sampler.weights.shape)  # Ensure the sampler gets the correct shape

# Apply the transforms to the datasets
train_dataset.dataset.transform = train_transforms
val_dataset.dataset.transform = val_transforms

# Create DataLoaders with the sampler for training and regular DataLoader for validation
# train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
train_loader = DataLoader(train_dataset, batch_size=32)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Number of classes: {len(class_weights_tensor)}")
print(f"Sample weights length: {len(sample_weights)}")
print(f"Sample weights: {sample_weights[:10]}")  # Show first 10 sample weights for inspection

Number of classes: 15
Unique labels in training set: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Class counts in training set: [ 798 1182  800  800  122 1701  800 1527  762 1417 1341 1123 2566  298
 1273]
Class weights: [0.05338422 0.03604112 0.05325076 0.05325076 0.34918529 0.02504445
 0.05325076 0.02789824 0.05590631 0.03006394 0.03176779 0.03793464
 0.01660195 0.14295505 0.03346473]
Sample weights length: 16510
Sampler weights shape: torch.Size([16510])
Number of classes: 15
Sample weights length: 16510
Sample weights: [0.03604112192988396, 0.02504444681107998, 0.027898235246539116, 0.053384218364953995, 0.03604112192988396, 0.01660194993019104, 0.01660194993019104, 0.03604112192988396, 0.05590630695223808, 0.03604112192988396]


In [11]:
# Import the Resnet model for transfer learning
model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
# Freeze the parameters in the base model
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
num_classes = len(class_counts)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Replace the last layer

# Move the model to the desired device (e.g., 'cuda' or 'cpu')
model = model.to(device)

In [ ]:
# Initialize the loss function with the class weights
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# Set up the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Adjust based on your needs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:  # Iterate over DataLoader
        images, labels = images.to(device), labels.to(device)  # Move to device

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)  # Calculate loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# Validation loop (optional)
model.eval()
with torch.no_grad():
    total_correct = 0
    total_samples = 0

    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {total_correct / total_samples:.4f}')

Epoch [1/10], Loss: 0.9256
Epoch [2/10], Loss: 0.3787
Epoch [3/10], Loss: 0.2599
Epoch [4/10], Loss: 0.1978
Epoch [5/10], Loss: 0.1580
Epoch [6/10], Loss: 0.1296
